# Import Required Libraries
Import the necessary libraries, including pandas and json.

In [36]:
# Import the necessary libraries, including pandas and json
import pandas as pd
import json
import os

In [44]:
# Define the path to the folder containing files
folder_path = 'Project/2023'

# Loop through all files in the folder
for filename in os.listdir(folder_path):
    if filename.startswith('2023') and not filename.endswith('.json'):
        old_file_path = os.path.join(folder_path, filename)
        new_file_path = os.path.join(folder_path, filename + '.json')
        
        # Rename the file
        os.rename(old_file_path, new_file_path)
        print(f"Renamed {old_file_path} to {new_file_path}")

print("Renaming completed.")

Renamed Project/2023\202300000 to Project/2023\202300000.json
Renamed Project/2023\202300001 to Project/2023\202300001.json
Renamed Project/2023\202300002 to Project/2023\202300002.json
Renamed Project/2023\202300003 to Project/2023\202300003.json
Renamed Project/2023\202300004 to Project/2023\202300004.json
Renamed Project/2023\202300005 to Project/2023\202300005.json
Renamed Project/2023\202300006 to Project/2023\202300006.json
Renamed Project/2023\202300007 to Project/2023\202300007.json
Renamed Project/2023\202300008 to Project/2023\202300008.json
Renamed Project/2023\202300009 to Project/2023\202300009.json
Renamed Project/2023\202300010 to Project/2023\202300010.json
Renamed Project/2023\202300011 to Project/2023\202300011.json
Renamed Project/2023\202300012 to Project/2023\202300012.json
Renamed Project/2023\202300013 to Project/2023\202300013.json
Renamed Project/2023\202300014 to Project/2023\202300014.json
Renamed Project/2023\202300015 to Project/2023\202300015.json
Renamed 

# Load JSON File
Load the JSON file from the specified path 'Project/2023/202302889.json'.

In [2]:
import os
import json
import pandas as pd

# Path to the folder containing JSON files
folder_path = 'Project/2023'

# Initialize a list to store the rows of the DataFrame
data_rows = []

# Columns to extract
columns_to_keep = [
    'coredata.dc:title',
    'coredata.prism:publicationName',
    'item.bibrecord.head.citation-info.citation-language.@language',
    'coredata.citedby-count'
]

# Helper function to safely extract a value from a dictionary
def safe_get(d, keys, default=None):
    try:
        for key in keys:
            d = d.get(key, {})
        return d if d else default
    except AttributeError:
        return default

# Iterate through each file in the folder
for file_name in os.listdir(folder_path):
    if file_name.startswith("2023") and file_name.endswith(".json"):
        file_path = os.path.join(folder_path, file_name)
        with open(file_path, 'r', encoding='utf-8') as file:
            try:
                data = json.load(file)

                # Extract countries from author-group
                author_groups = safe_get(data, ['abstracts-retrieval-response', 'item', 'bibrecord', 'head', 'author-group'], [])
                countries = [
                    safe_get(author, ['affiliation', 'country'], 'Unknown')
                    for author in author_groups
                ]
                countries_string = ','.join(countries)

                # Extract keywords
                auth_keywords = safe_get(data, ['abstracts-retrieval-response', 'authkeywords', 'author-keyword'], [])
                keywords = [keyword.get('$', '') for keyword in auth_keywords if isinstance(keyword, dict)]
                keywords_string = ','.join(keywords)

                # Normalize JSON data and filter columns
                row_data = pd.json_normalize(data.get('abstracts-retrieval-response', {}))
                row = {col: row_data[col].iloc[0] if col in row_data else pd.NA for col in columns_to_keep}

                # Add processed fields
                row['item.bibrecord.head.author-group.affiliation.country'] = countries_string
                row['authkeywords.author-keyword'] = keywords_string

                # Append the row to data_rows
                data_rows.append(row)

            except Exception as e:
                print(f"Error processing file {file_name}: {e}")

# Convert the list of rows into a DataFrame
df = pd.DataFrame(data_rows)

# Save the DataFrame to a CSV file
output_path = 'output.csv'
df.to_csv(output_path, index=False, encoding='utf-8')

# Display the DataFrame for verification
print(df)


                                      coredata.dc:title  \
0     Graphene oxide-alginate hydrogel-based indicat...   
1     Rare coordination behavior of triethanolamine ...   
2     Total ammonia nitrogen removal and microbial c...   
3     Effects of microaeration and sludge recirculat...   
4     Bioaccumulation of heavy metals in commerciall...   
...                                                 ...   
2885  Long-chain bio-olefins production via oxidativ...   
2886  Recent Developments and Applications of Microf...   
2887  Social justice, education and peacebuilding: c...   
2888  Effects of black soldier fly (Hermetia illucen...   
2889  Effects of remittances on household poverty an...   

                         coredata.prism:publicationName  \
0     International Journal of Biological Macromolec...   
1                        Journal of Molecular Structure   
2                                           Aquaculture   
3                      Science of the Total Environment

In [55]:
import json
import pandas as pd

# Define the path to the JSON file
json_file_path = 'Project/2023/202302889.json'

# Open and load the JSON file
with open(json_file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

# Extract countries from author-group
countries = [author['affiliation']['country'] for author in data['abstracts-retrieval-response']['item']['bibrecord']['head']['author-group']]

# Convert the list of countries to a single string
countries_string = ','.join(countries)

# Extract $ values from authkeywords.author-keyword
keywords = [keyword['$'] for keyword in data['abstracts-retrieval-response']['authkeywords']['author-keyword']]

# Convert the list of keywords to a single string
keywords_string = ','.join(keywords)

# Normalize JSON data to a DataFrame
df = pd.json_normalize(data['abstracts-retrieval-response'])

# Define columns to keep
columns_to_keep = [
    'coredata.dc:title',
    'coredata.prism:publicationName',
    'item.bibrecord.head.citation-info.citation-language.@language',
    'coredata.citedby-count'
]

# Add the countries_string and keywords_string to the DataFrame
df['item.bibrecord.head.author-group.affiliation.country'] = countries_string
df['authkeywords.author-keyword'] = keywords_string

# Keep only the specified columns
df = df[columns_to_keep + ['item.bibrecord.head.author-group.affiliation.country', 'authkeywords.author-keyword']]


df.to_csv('output.csv', index=False)

# Display the DataFrame to verify the contents
df

,coredata.dc:title,coredata.prism:publicationName,item.bibrecord.head.citation-info.citation-language.@language,coredata.citedby-count,item.bibrecord.head.author-group.affiliation.country,authkeywords.author-keyword
0,Effects of remittances on household poverty an...,Journal of the Asia Pacific Economy,English,3,"Cambodia,Thailand","Cambodia,consumption,expenditure,Impact,inequa..."
